In [1]:
import cv2
import keras
import numpy as np
import tensorflow

Using TensorFlow backend.


In [2]:
kamera = cv2.VideoCapture(0) #Untuk melakukan video capture, 0=kamera default 1=kamera external

model = keras.models.load_model("MNIST.model") #Untuk lood model

kernel = np.ones(5)

font = cv2.FONT_HERSHEY_SIMPLEX #Font untuk label ketika melakukan videocapture

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [3]:
label = {
    0 : "Nol",
    1 : "Satu",
    2 : "Dua",
    3 : "Tiga",
    4 : "Empat",
    5 : "Lima",
    6 : "Enam",
    7 : "Tujuh",
    8 : "Delapan",
    9 : "Sembilan"
}


In [4]:
def nothing(x):
    pass

#Memberi nama pada windows
cv2.namedWindow('Pengatur')

#Isi windows H=hue S=saturation, V=Value, Th=Threshold, min=batas bawah, max=batas atas
cv2.createTrackbar('H_Min', 'Pengatur',0,255,nothing)
cv2.createTrackbar('S_Min', 'Pengatur',0,255,nothing)
cv2.createTrackbar('V_Min', 'Pengatur',0,255,nothing)
cv2.createTrackbar('H_Max', 'Pengatur',0,255,nothing)
cv2.createTrackbar('S_Max', 'Pengatur',0,255,nothing)
cv2.createTrackbar('V_Max', 'Pengatur',0,255,nothing)
cv2.createTrackbar('Th_Min', 'Pengatur',0,255,nothing)
cv2.createTrackbar('Th_Max', 'Pengatur',0,255,nothing)

#try untuk menguji error
try:
    while(True):
        _, frame = kamera.read()
        
        #Inisialisasi frame HSV convert dari RGB ke HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        #Untuk membuatt 1 frame pesergi panjang 100x300 (buat kotak frame)
         # Draw a box cv2.rectangle(flame,(left,right),(top,bottom),(b,g,r),ketebalan)
        cv2.rectangle(frame,(100,100),(300,300),(255,255,255),1)
        
        #Untuk membuat windows berisi frame RGB dengan ukuran 100x300 (hasil crop RGB)
        gambar_crop_hsv = hsv[100:300, 100:300]
        #Untuk membuat windows berisi frame HSV dengan ukuran 100x300 (hasil crop frame)
        gambar_crop = frame[100:300, 100:300]
        
        #Untuk mendapatkan Value dari trackbar
        H_Min = cv2.getTrackbarPos('H_Min','Pengatur')
        S_Min = cv2.getTrackbarPos('S_Min','Pengatur')
        V_Min = cv2.getTrackbarPos('V_Min','Pengatur')
        H_Max = cv2.getTrackbarPos('H_Max','Pengatur')
        S_Max = cv2.getTrackbarPos('S_Max','Pengatur')
        V_Max = cv2.getTrackbarPos('V_Max','Pengatur')
        Th_Min = cv2.getTrackbarPos('Th_Min','Pengatur')
        Th_Max = cv2.getTrackbarPos('Th_Max','Pengatur')
        
        batas_bawah = np.array([H_Min,S_Min,V_Min])
        batas_atas = np.array([H_Max,S_Max,V_Max])
        
        #Gambar crop pada Kamera nanti akan ditentukan batas bawah dan batas atassnya
        mask = cv2.inRange(gambar_crop_hsv, batas_bawah, batas_atas)
        
        #Selanjutnya Gambar Crop akan dihitung nilai bit dari gambar crop tadi
        hasil = cv2.bitwise_and(gambar_crop, gambar_crop, mask = mask)
        
        #Gambar Hasil bitwite tadi selanjutnya diconvert menjadi gambar grayscale
        hasil = cv2.cvtColor(hasil, cv2.COLOR_BGR2GRAY)
        
        #Gambar Grayscale tadi selanjutnya di tentuin nilai treshold biner minimum dan maximum
        _, hasil_threshold = cv2.threshold(hasil, Th_Min, Th_Max, cv2.THRESH_BINARY)
        
        #Selanjutnya hasil threshold yang sudah diubah kebiner, Binernya diperjelas atau diperbesar
        hasil_threshold_dilasi = cv2.dilate(hasil_threshold, kernel, iterations = 2)
        
        gambar_resize = cv2.resize(hasil_threshold_dilasi,(28,28))
        
        #print("Dimensi Gambar Resize : ", gambar_resize.shape)
        
        gambar_reshape = np.array(gambar_resize).reshape(1,28,28)
        
        #Melakukan Prediksi
        prediksi = model.predict(x=gambar_reshape)
        label_prediksi = np.argmax(prediksi)
        
        #Meletakkan label hasil prediksi pada frame
        cv2.putText(frame,str(label[label_prediksi]),(50,50),font,2,(255,255,0),2,cv2.LINE_AA)
        
        cv2.imshow('Output Crop', hasil_threshold_dilasi)
        
        cv2.imshow('Frame', frame)
        
        cv2.imshow('Hasil Crop', gambar_crop)
        
        if((cv2.waitKey(1) & 0xFF) == ord('q')):
            kamera.release()
            cv2.destroyAllWindows()
            
except KeyboardInterrupt:
    kamera.release()
    cv2.destroyAllWindows()